# MLP para Predição de Indicadores Sociais

### Importação das bibliotecas

In [4]:
import pandas as pd
import numpy as pd
import xlrd
import torch
import torch.nn as nn
import torch.optim as optim

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler